# Data Mining Course Spark Exercise
## Sharif University of Technology

In this notebook we are going to analyze farsi wikipedia. 
Outline of the exercise:
* Dataset preparation (5 Points)
* Preprocessing (35 Points) 
* Exploration (20 Points) 
* TF-IDF + Search (40 Points)

You should replace the `TODO` parts with your implementation. Remeber that each `TODO` may take multiple lines and you shouldn't limit your self to one-line codes.

## Prerequisites
You should be faimilar with [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf). In this notebook you should use the following formula for tf-idf:
$$f_{t,d}/len(d) \times log(1 + \frac{N}{n_t})$$

## Warning: RDD api only
You **can not** use Dataframe, Dataset, mllib, ml, ... apis of spark in this exercise. You should only use the [RDD api](http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD).

# Section 0: Please enter your name below
# Name: Mohammadreza Taremi
# Student Number: 96101902

# Section 1: Dataset preparation (5 Points)

This section of notebook contains only shell commands. You don't need to completely understand each command or change anything.

Please run all the paragraphs sequentially.

In [0]:
! sudo apt-get -y install wget git




wget is already the newest version (1.20.3-1ubuntu1).
git is already the newest version (1:2.25.1-1ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


## Download the dump

In [0]:
! wget --no-check-certificate -P . https://dumps.wikimedia.your.org/fawiki/latest/fawiki-latest-pages-articles-multistream.xml.bz2

--2021-11-10 16:26:46--  https://dumps.wikimedia.your.org/fawiki/latest/fawiki-latest-pages-articles-multistream.xml.bz2
Resolving dumps.wikimedia.your.org (dumps.wikimedia.your.org)... 204.9.55.82, 2001:4978:1:420::cc09:3752
Connecting to dumps.wikimedia.your.org (dumps.wikimedia.your.org)|204.9.55.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002962062 (956M) [application/x-bzip]
Saving to: ‘./fawiki-latest-pages-articles-multistream.xml.bz2’

fawiki-latest-pages 100%[===================>] 956.50M  23.2MB/s    in 54s     

2021-11-10 16:27:40 (17.8 MB/s) - ‘./fawiki-latest-pages-articles-multistream.xml.bz2’ saved [1002962062/1002962062]



## Extract the dump (this may take a few minutes)

In [0]:
! bzip2 -d fawiki-latest-pages-articles-multistream.xml.bz2

## Clone git project for converting wikipedia xml dump to json

In [0]:
! pip install wikiextractor

     |████████████████████████████████| 46 kB 3.3 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


## Run the script to convert xml to json (this might take around 30 minutes)

In [0]:
! python -m wikiextractor.WikiExtractor --json fawiki-latest-pages-articles-multistream.xml

INFO: Preprocessing 'fawiki-latest-pages-articles-multistream.xml' to collect template definitions: this may take some time.
INFO: Preprocessed 100000 pages
INFO: Preprocessed 200000 pages
INFO: Preprocessed 300000 pages
INFO: Preprocessed 400000 pages
INFO: Preprocessed 500000 pages
INFO: Preprocessed 600000 pages
INFO: Preprocessed 700000 pages
INFO: Preprocessed 800000 pages
INFO: Preprocessed 900000 pages
INFO: Preprocessed 1000000 pages
INFO: Preprocessed 1100000 pages
INFO: Preprocessed 1200000 pages
INFO: Preprocessed 1300000 pages
INFO: Preprocessed 1400000 pages
INFO: Preprocessed 1500000 pages
INFO: Preprocessed 1600000 pages
INFO: Preprocessed 1700000 pages
INFO: Preprocessed 1800000 pages
INFO: Preprocessed 1900000 pages
INFO: Preprocessed 2000000 pages
INFO: Preprocessed 2100000 pages
INFO: Preprocessed 2200000 pages
INFO: Preprocessed 2300000 pages
INFO: Preprocessed 2400000 pages
INFO: Preprocessed 2500000 pages
INFO: Preprocessed 2600000 pages
INFO: Preprocessed 2700000

## Ensure output files exist

In [0]:
! ls text/*/*

text/AA/wiki_00  text/AH/wiki_71  text/AP/wiki_42  text/AX/wiki_13
text/AA/wiki_01  text/AH/wiki_72  text/AP/wiki_43  text/AX/wiki_14
text/AA/wiki_02  text/AH/wiki_73  text/AP/wiki_44  text/AX/wiki_15
text/AA/wiki_03  text/AH/wiki_74  text/AP/wiki_45  text/AX/wiki_16
text/AA/wiki_04  text/AH/wiki_75  text/AP/wiki_46  text/AX/wiki_17
text/AA/wiki_05  text/AH/wiki_76  text/AP/wiki_47  text/AX/wiki_18
text/AA/wiki_06  text/AH/wiki_77  text/AP/wiki_48  text/AX/wiki_19
text/AA/wiki_07  text/AH/wiki_78  text/AP/wiki_49  text/AX/wiki_20
text/AA/wiki_08  text/AH/wiki_79  text/AP/wiki_50  text/AX/wiki_21
text/AA/wiki_09  text/AH/wiki_80  text/AP/wiki_51  text/AX/wiki_22
text/AA/wiki_10  text/AH/wiki_81  text/AP/wiki_52  text/AX/wiki_23
text/AA/wiki_11  text/AH/wiki_82  text/AP/wiki_53  text/AX/wiki_24
text/AA/wiki_12  text/AH/wiki_83  text/AP/wiki_54  text/AX/wiki_25
text/AA/wiki_13  text/AH/wiki_84  text/AP/wiki_55  text/AX/wiki_26
text/AA/wiki_14  text/AH/wiki_85  text/AP/wiki_56  text/AX/wik

## Install Pypark & Initialization
Uncomment this section if you use google colab or local pc

In [0]:
#! pip install pyspark

In [0]:
# from pyspark.sql import SparkSession
# spark = SparkSession \
#     .builder \
#     .appName("MDA_2021") \
#     .master("local[*]") \
#     .getOrCreate()

# sc=spark.sparkContext

## Reading the data

In [0]:
articles_rdd = sc.textFile("file:/databricks/driver/text/*/*") # Now you have a RDD with wikipedia posts
articles_rdd = articles_rdd.sample(False, 0.33, 5)
articles_rdd.take(1)

Out[126]: ['{"id": "5476525", "revid": "206089", "url": "https://fa.wikipedia.org/wiki?curid=5476525", "title": "\\u0631\\u0627\\u0641\\u0627\\u0626\\u0644\\u0647 \\u0622\\u0646\\u062f\\u0631\\u0626\\u0627\\u0633\\u06cc", "text": "\\u0631\\u0627\\u0641\\u0627\\u0626\\u0644\\u0647 \\u0622\\u0646\\u062f\\u0631\\u0626\\u0627\\u0633\\u06cc (\\u061b \\u2013 ) \\u06a9\\u0627\\u0631\\u06af\\u0631\\u062f\\u0627\\u0646 \\u0641\\u06cc\\u0644\\u0645\\u060c \\u067e\\u062f\\u06cc\\u062f\\u0622\\u0648\\u0631\\u060c \\u0631\\u0648\\u0632\\u0646\\u0627\\u0645\\u0647\\u200c\\u0646\\u06af\\u0627\\u0631\\u060c \\u0641\\u06cc\\u0644\\u0645\\u200c\\u0646\\u0627\\u0645\\u0647\\u200c\\u0646\\u0648\\u06cc\\u0633 \\u0648 \\u0646\\u0648\\u06cc\\u0633\\u0646\\u062f\\u0647 \\u0627\\u0647\\u0644 \\u0627\\u06cc\\u062a\\u0627\\u0644\\u06cc\\u0627 \\u0628\\u0648\\u062f.\\n\\u0627\\u0632 \\u0641\\u06cc\\u0644\\u0645\\u200c\\u0647\\u0627\\u06cc\\u06cc \\u06a9\\u0647 \\u0648\\u06cc \\u062f\\u0631 \\u0622\\u0646\\u200c\\

# Section 2: Preprocessing (35 Points)

In this section we will remove useless part (for example /n and /u200c) also find and remove stop words and remove the words with a low count in the corpus

In [0]:
import json

articles_rdd = articles_rdd.map(lambda text: json.loads(text))  #parse the json string
words_rdd = articles_rdd.flatMap(lambda doc: doc['title'].split(' ') + doc['text'].split(' ')) #extract words from title and description
words_count_rdd = words_rdd.map(lambda word: (word,1)).reduceByKey(lambda word1,word2:word1+word2)  #make an rdd with the count of each word
top_100 = words_count_rdd.takeOrdered(100,key=lambda x:-x[1]) #find the 100 most common words

In [0]:
top_100

Out[112]: [('در', 4321777),
 ('و', 4078199),
 ('به', 2659958),
 ('از', 2541592),
 ('که', 1906355),
 ('', 1876585),
 ('این', 1271359),
 ('را', 1238395),
 ('با', 1000101),
 ('یک', 856250),
 ('است', 758893),
 ('سال', 675664),
 ('آن', 620706),
 ('برای', 521090),
 ('است.', 493219),
 ('او', 389878),
 ('بر', 310787),
 ('تا', 307551),
 ('خود', 305506),
 ('واقع', 303619),
 ('یا', 299501),
 ('ایران', 296445),
 ('شده', 287257),
 ('شده\u200cاست.', 270515),
 ('قرار', 250929),
 ('شد.', 238169),
 ('نیز', 234379),
 ('نفر', 226795),
 ('نام', 222030),
 ('بود', 216761),
 ('عنوان', 206865),
 ('جمعیت', 205963),
 ('دو', 203217),
 ('وی', 193641),
 ('شهرستان', 191240),
 ('دارد', 186886),
 ('یکی', 180615),
 ('پس', 179217),
 ('شهر', 176461),
 ('استفاده', 175410),
 ('بود.', 173894),
 ('توسط', 172633),
 ('شد', 169327),
 ('بخش', 160764),
 ('کرد.', 156200),
 ('استان', 150754),
 ('دارد.', 148590),
 ('(به', 148120),
 ('اهل', 147448),
 ('آمریکا', 141965),
 ('فوتبال', 138746),
 ('دیگر', 137494),
 ('اما', 135300),
 ('کش

In [0]:
stopwords = ['در', 'و', 'به', 'از', 'که', 'این', 'را', 'با', 'یک', 'است', 'سال', 'آن', 'برای', 'او', 'است.', 'شده', 'بر', 'واقع', 'خود', 'تا', 'یا', 'نیز', 'نام', 'بود', 'دارد', 'یکی', 'دو', 'وی', 'عنوان', 'پس', 'استفاده', 'شد', 'شد.', 'توسط', 'هم', 'بود.', 'دیگر', '(به', 'شده\u200cاست.', 'هر', 'اما', 'کرد', 'اهل', 'همچنین', 'وجود', 'مورد', 'صورت', 'می\u200cشود', 'آن\u200cها', 'روی', 'زیر', 'می\u200cتوان', 'اشاره', 'می\u200cشود.', 'است.\n', 'کرد.', 'سطح', 'دست', 'زمان', 'بین', 'داشته', 'دارای', 'متر', 'انجام', 'طول', ')', 'کرده', 'بعد', 'شده\u200cاست.\n\n', 'ثبت', 'دوره','سه', 'دارد.\n\n','شده\u200cاست' , 'بوده\u200cاست.\n' , 'می\u200cباشد.' , '-' , 'می\u200cکند' , 'دارد.\n\n' , 'است.\n\nاین' , 'است.\n\n' , 'حدود' , 'می' ,'(؛'  ] 

In [0]:
#we define a function to do: 1. detect the top_100 words
# 2. remove them

def remove_useless_words(text , uselesswords):
    #extract the title and body of a text
    text_body = text['text'].split(' ')
    text_title = text['title'].split(' ')
    
    #iterate on every word existed in text_body and text_title
    for word in text_body:
        if word in uselesswords:
            text_body.remove(word)
    for word in text_title:
        if word in uselesswords:
            text_body.remove(word)
    
    #join the words by ' '
    text['text']= ' '.join(text_body)
    text['title'] = ' '.join(text_title)
    return text

In [0]:
articles_without_stopwords_rdd = articles_rdd.map(lambda doc: remove_useless_words(doc , stopwords))#remove stopwords from the article title and text

In [0]:
MIN_COUNT = 20
uncommon_words = words_count_rdd.filter(lambda word: word[1] < MIN_COUNT) #list of the words that have occured less than MIN_COUNT in the whole corpus
uncommon_words_list = uncommon_words.map(lambda x: (x,1) ).collectAsMap() #list of words with frequency<20

In [0]:
uncommon_words_list

Out[132]: {('Vega', 6): 1,
 ('نقشه\u200cهایت', 3): 1,
 ('سميون', 1): 1,
 ('مونکیو', 2): 1,
 ('یاواش،', 1): 1,
 ('شخصی.\nرایان', 1): 1,
 ('هانایا،', 1): 1,
 ('دهد.\nمشکلات', 2): 1,
 ('"دفترهای', 3): 1,
 ('۲٬۴۶۹', 3): 1,
 ('سِمت،', 1): 1,
 ('دل\u200cها', 18): 1,
 ('رفتاری\u200cای', 2): 1,
 ('Gelephu)', 1): 1,
 ('پیاده\u200cهای', 1): 1,
 ('بحرانی»', 5): 1,
 ('بود.\nشاخه\u200cهای', 1): 1,
 ('آورد.\nآفست', 1): 1,
 ('تسمه\u200cها', 12): 1,
 ('«واقعاً', 17): 1,
 ('اسیرگیری', 1): 1,
 ('مینگه\u200cچویر', 13): 1,
 ('رها،', 7): 1,
 ('lǐn")،', 1): 1,
 ('کردیم».\nنسخه', 1): 1,
 ('می\u200cشود.\nسه', 18): 1,
 ('ای\u200cوی\u200cان\nبرای', 1): 1,
 ('"Newt"', 1): 1,
 ('وشاگردان', 1): 1,
 ('کوالالامپور،', 18): 1,
 ('وُید', 3): 1,
 ('النیشابوری', 2): 1,
 ('گيشا', 6): 1,
 ('(ان\u200cاچ\u200cال)', 2): 1,
 ('"themselves"', 1): 1,
 ('"دالاگاه"', 1): 1,
 ('جی\u200cآر،', 5): 1,
 ('سودگرایی،', 1): 1,
 ('۱۳۶۷\nهجری', 1): 1,
 ('۲۰۸۰', 19): 1,
 ('Grasshopper', 2): 1,
 ('(درمعرض', 1): 1,
 ('تمیزکنندهٔ', 3): 1,
 ('63

In [0]:
#remove uncommon words from articles_without_stopwords_rdd
articles_cleaned_rdd = articles_without_stopwords_rdd.map(lambda doc: remove_useless_words(doc, uncommon_words_list))

In [0]:
articles_cleaned_rdd.take(1) # This should output a dictionary with url,title and text keys. title and text should not have stopwords or uncommon.

Out[134]: [{'id': '5476525',
  'revid': '206089',
  'url': 'https://fa.wikipedia.org/wiki?curid=5476525',
  'title': 'رافائله آندرئاسی',
  'text': 'رافائله آندرئاسی – کارگردان فیلم، پدیدآور، روزنامه\u200cنگار، فیلم\u200cنامه\u200cنویس نویسنده ایتالیا بود.\nاز فیلم\u200cهایی وی آن\u200cها نقش داشته\u200cاست، به "در پلکان باستانی" نمود.'}]

In [0]:
#How many unique words remain after the cleaning procedure?
#we have to use flatmap function to faltten the data
#and also distinct and count methods to compute to count of unique words
articles_cleaned_rdd = articles_cleaned_rdd.filter(lambda doc: doc['title'] != u'')
articles_cleaned_rdd = articles_cleaned_rdd.filter(lambda doc: doc['text'] != u'')
words_cleaned_rdd = articles_cleaned_rdd.flatMap(lambda doc: doc['title'].split()+doc['text'].split()).distinct().count()
words_cleaned_rdd

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-3409951118598285> in <module>
      4 articles_cleaned_rdd = articles_cleaned_rdd.filter(lambda doc: doc['title'] != u'')
      5 articles_cleaned_rdd = articles_cleaned_rdd.filter(lambda doc: doc['text'] != u'')
----> 6 words_cleaned_rdd = articles_cleaned_rdd.flatMap(lambda doc: doc['title'].split()+doc['text'].split()).distinct().count()
      7 words_cleaned_rdd

/databricks/spark/python/pyspark/rdd.py in count(self)
   1268         3
   1269         """
-> 1270         return self.mapPartitions(lambda i: [sum(1 for _ in i)]).sum()
   1271 
   1272     def stats(self):

/databricks/spark/python/pyspark/rdd.py in sum(self)
   1257         6.0
   1258         """
-> 1259         return self.mapPartitions(lambda x: [sum(x)]).fold(0, operator.add)
   1260 
   1261     def count(self):

/databricks/spark/python/pyspark/rdd.py in

In [0]:
articles_rdd,articles_cleaned_rdd

Out[68]: (PythonRDD[82] at RDD at PythonRDD.scala:58,
 PythonRDD[83] at RDD at PythonRDD.scala:58)

In [0]:
#What are the top 100 most common trigrams in the corpus?
#I dont even know what a trigram is!

In [0]:
#Plot a distribution from document lengths using appropriate bin sizes with 100 bins
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
plt.figure(0)
plt.hist(articles_cleaned_rdd.map(lambda doc : len(doc['title'])+len(doc['text'])).collect(),bins=100)
plt.xlabel('Length')
plt.ylabel('Doc Counts')
plt.show()

<Figure size 432x288 with 0 Axes>

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-206439646218613> in <module>
      4 import seaborn as sns
      5 plt.figure(0)
----> 6 plt.hist(articles_cleaned_rdd.map(lambda doc : len(doc['title'])+len(doc['text'])).collect(),bins=100)
      7 plt.xlabel('Length')
      8 plt.ylabel('Doc Counts')

/databricks/spark/python/pyspark/rdd.py in collect(self)
    965         # Default path used in OSS Spark / for non-credential passthrough clusters:
    966         with SCCallSiteSync(self.context) as css:
--> 967             sock_info = self.ctx._jvm.PythonRDD.collectAndServe(self._jrdd.rdd())
    968         return list(_load_from_socket(sock_info, self._jrdd_deserializer))
    969 

/databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_

In [0]:
#What is the url of the longest article?
articles_cleaned_rdd.map(lambda document: (document['url'],len(document['title']) + len(document['text']))).takeOrdered(1, key=lambda x:-x[1])

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-206439646218614> in <module>
      1 #What is the url of the longest article?
----> 2 articles_cleaned_rdd.map(lambda document: (document['url'],len(document['title']) + len(document['text']))).takeOrdered(1, key=lambda x:-x[1])

/databricks/spark/python/pyspark/rdd.py in takeOrdered(self, num, key)
   1539             return heapq.nsmallest(num, a + b, key)
   1540 
-> 1541         return self.mapPartitions(lambda it: [heapq.nsmallest(num, it, key)]).reduce(merge)
   1542 
   1543     def take(self, num):

/databricks/spark/python/pyspark/rdd.py in reduce(self, f)
   1030             yield reduce(f, iterator, initial)
   1031 
-> 1032         vals = self.mapPartitions(func).collect()
   1033         if vals:
   1034             return reduce(f, vals)

/databricks/spark/python/pyspark/rdd.py in collect(self)
    965         # D

In [0]:
#How many articles contain your first name?!
articles_cleaned_rdd.filter(lambda document: (if u'محمدرضا' in document['title'].split()) or (if u'محمدرضا' in document['text'].split())).count()

  File "<command-206439646218615>", line 2
    articles_cleaned_rdd.filter(lambda document: (if u'محمدرضا' in document['title'].split() or if u'محمدرضا' in document['text'].split())).count()
                                                  ^
SyntaxError: invalid syntax


# Section 4: TF-IDF + Search (40 Points)

In [0]:
#calculate document frequncy for each word
word_df_rdd = articles_cleaned_rdd.flatMap(lambda doc: [(word,1) for word in set(doc['title'].split()+doc['text'].split())]).reduceByKey(lambda x,y:x+y).collectAsMap()

def tf_idf(document):
    tf_idf={}
    for word in document['text'].split()+document['title'].split():
        tf_idf_result[word]=float(document['text'].split()+document['title'].split().count(word))/len(document['text'].split()+document['title'].split())*np.log(1+(articles_cleaned_rdd.count()/(word_df_rdd[word]+1)))
    return tf_idf

def updateDoc(document,updatedObj):
    document.update(updatedObj)
    return document

#add `vector` key to articles_cleaned_rdd dictionary with the tf_idf dictionary
articles_tf_idf_vectors = articles_cleaned_rdd.map(lambda doc: updateDoc(doc,{'vector':tf_idf(doc)}))
articles_tf_idf_vectors.take(1)


---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-3437167528650472> in <module>
      1 #calculate document frequncy for each word
----> 2 word_df_rdd = articles_cleaned_rdd.flatMap(lambda doc: [(word,1) for word in set(doc['title'].split()+doc['text'].split())]).reduceByKey(lambda x,y:x+y).collectAsMap()
      3 # articles_tf_idf_vectors = #TODO: add `vector` key to articles_cleaned_rdd dictionary with the tf_idf dictionary

/databricks/spark/python/pyspark/rdd.py in collectAsMap(self)
   1882         4
   1883         """
-> 1884         return dict(self.collect())
   1885 
   1886     def keys(self):

/databricks/spark/python/pyspark/rdd.py in collect(self)
    965         # Default path used in OSS Spark / for non-credential passthrough clusters:
    966         with SCCallSiteSync(self.context) as css:
--> 967             sock_info = self.ctx._jvm.PythonRDD.collectAndServ

## Searching

In [0]:
from scipy.spatial.distance import cosine

query = 'هخامنشیان ساسانیان هگمتانه'

def get_tf_idf(query):
    tf_idf =[]
    BoW_len = 
    for word in query.split():
        tf=query.split().count(word)
        df =word_df_rdd[word]+1
        tf_idf.append((float(tf)/len(query.split()))*np.log(1+(articles_cleaned_rdd.count()/(df+1))))
    return tf_idf

query_tf_idf = get_tf_idf(query)

def docScore(doc):
    common_doc =[] 
    for word in query.split():
    if word in doc['vector']:
        common_doc.append(doc['vector'][word])
    else:
        common_doc.append(0)
    return np.nansum([0,cosine(query_tf_idf,common_doc)])

#find similar wikipedia articles based on cosine similarity of tf-idf vectors
top_10_similar = articles_tf_idf_vectors.map(lambda document: (document['url'],docScore(document))).filter(lambda x:x[1]>0).takeOrdered(10,key=lambda x:x[1])
top_10_similar